In [1]:
%run env_setup.py
%matplotlib notebook
import importlib

In [2]:
import os
import torch
from torch import FloatTensor, nn
from torch.autograd import Variable
import torchvision
import yama
import yama.vision as yamavision
from tqdm import tqdm_notebook as tqdm

In [3]:
from yama.vision.datasets import LocalStorage, PaperSpaceGradientStorage

In [4]:
storage = LocalStorage(os.path.abspath('../../_data'))
#storage = PaperSpaceGradientStorage()

In [6]:
batch_size, image_size, noise_size = 256, (64, 64), 100

In [12]:
%env http_proxy=http://127.0.0.1:1087
%env https_proxy=http://127.0.0.1:1087

env: http_proxy=http://127.0.0.1:1087
env: https_proxy=http://127.0.0.1:1087


In [7]:
data_path = os.path.abspath('../../_data/cifar10')

data = torchvision.datasets.CIFAR10(root=data_path, download=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.Resize(image_size),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
)

Files already downloaded and verified


In [ ]:
#data_path = os.path.abspath('../../_data/lsun/')
#data_path = '../../_data/lsun'
data = yama.vision.datasets.LSUN(storage, classes=['bedroom_train'],
    transform=torchvision.transforms.Compose([
        torchvision.transforms.Resize(image_size),
        torchvision.transforms.CenterCrop(image_size),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
)

In [26]:
dataloader = torch.utils.data.DataLoader(data, batch_size, shuffle=True, num_workers=os.cpu_count())
n = len(dataloader); n
sample_x, sample_y = data[0]
input_size = tuple(sample_x.size())

In [12]:
class DCGan_D(nn.Module):
    def __init__(self, input_size):
        img_channels, img_size = input_size[0], input_size[1:]
        main = nn.Sequential()

        def conv_block(name, in_channels, out_channels, kernel_size):
            main.add_module('Conv-{name}-{in_channels}-{out_channels}-{kernel_size}'.format(*locals()),
                           nn.Conv2d(in_channels, out_channels, kernel_size))
            main.add_module('BN-{name}'.format(*locals()),
                           nn.BatchNorm2d(out_channels))
            main.add_module('LeakyRelu-{name}'.format(*locals()),
                           nn.LeakyReLU())
        
        conv_block('b1', img_channels, )
    
    def forward(self, imgs):
        
        

class DCGan_G(nn.Module):
    def __init__(self, noise_len):
        pass

def noise(shape):
    return None

def make_trainable(m, b=True):
    for v in m.parameters(): v.require_grad = b
    
def train(D, G, opt_D, opt_G, loader, epochs, batch_size, first=True):
    
    n = len(loader)
    make_trainable(D)
    for ep in range(epochs):
        gen_iter = 0
        d_iter = 0
        d_iter_tgt = 0
        bar = tqdm(loader)
        for real in bar:
            if d_iter_tgt == 0 or d_iter >= d_iter_tgt:
                is_warm_up = first and gen_iter < 25
                if is_warm_up or gen_iter % 500 == 0:
                    d_iter_tgt = 100
                else:
                    d_iter_tgt = 5
                d_iter = 0
            fake = G(noise(real.size()[0])).data
            err = D(real) - D(fake)
            err.backward()
            opt_D.step()
            bar.set_postfix(loss=float(err.data))
            
            d_iter += 1
            if d_iter == d_iter_tgt:
                make_trainable(D, False)
                fake = G(noise(batch_size))
                D(fake).backward()   # Generator want to cheat D -> 0 (means real)
                opt_G.step()
                gen_iter += 1
                make_trainable(D, True)

IndentationError: expected an indented block (<ipython-input-12-a2d3b6c63623>, line 9)

In [11]:
g = DCGan_G()
d = DCGan_D()

opt_d = torch.optim.RMSprop(d.parameters())
opt_g = torch.optim.RMSprop(g.parameters())

train(d, g, opt_d, opt_g, dataloader,
      epochs=100, batch_size=32)

TypeError: 'method' object is not iterable